# S6's Hello, World!

A simple illustration of how S6 decorators can be used to produce a JIT-compiled function, using a very simple function.

Decorate the function so that it can be JIT-compiled.

In [1]:
import s6

@s6.jit
def hello(x):
  return f"Hello, {x}"

Time the function before and after optimization. To make sure the function is optimized/compiled, invoke it many times.

In [2]:
%time hello("World")

CPU times: user 320 µs, sys: 28 µs, total: 348 µs
Wall time: 354 µs


'Hello, World'

In [3]:
for i in range(200000):
    hello(str(i))
%time hello("World!")

CPU times: user 10 µs, sys: 1 µs, total: 11 µs
Wall time: 15.3 µs


'Hello, World!'

As you can see the compiled function runs much faster.

Now we can look at the strongjit from the compiled function, which is S6's own IR that is created from the Python bytecode version of the function.

In [4]:
print(s6.inspect(hello).strongjit)

function hello {
&0: [ %1 ]                                                  // entry point
  bytecode_begin @0 fastlocals [%1]                         // LOAD_CONST 1174978594.py:5
  %3 = frame_variable consts, $1
  incref notnull %3
  deoptimize_if_safepoint not %1, @2 stack [%3] fastlocals [%1], "Fastlocal load was nullptr (unbound local)" // LOAD_FAST 1174978594.py:5
  incref notnull %1
  %7 = constant $0
  %8 = call_native PyObject_Format (%1, %7) @4              // FORMAT_VALUE 1174978594.py:5
  decref notnull %1 @4                                      // FORMAT_VALUE 1174978594.py:5
  %10 = constant $0
  %11 = cmp eq i64 %8, %10
  deoptimize_if %11, &34, &13, materializing values [%1, %3]

&13:                                                        // preds: &0
  bytecode_begin @6 stack [%3, %8] fastlocals [%1]          // BUILD_STRING 1174978594.py:5
  %15 = constant $0
  %16 = call_native PyUnicode_New (%15, %15) @6             // BUILD_STRING 1174978594.py:5
  %17 = constant 

Finally we can look at the x86 assembly that is generated from the strongjit IR:

In [5]:
print(s6.inspect(hello).x86asm)

0x7f3ceb7ff000: push rbp
0x7f3ceb7ff001: mov rbp, rsp
0x7f3ceb7ff004: sub rsp, 0x70
0x7f3ceb7ff008: mov [rbp-0x60], r12
0x7f3ceb7ff00c: mov [rbp-0x58], rbx
0x7f3ceb7ff010: mov r12, rsi
0x7f3ceb7ff013: mov rbx, rdi
0x7f3ceb7ff016: lea rsi, [rbp-0x28]
0x7f3ceb7ff01a: mov rdx, 0x7f3cf801ce30
0x7f3ceb7ff024: call 0x7f3d243374b0
0x7f3ceb7ff02a: mov rsi, [rbx+0x168]
0x7f3ceb7ff031: mov qword [rbx+0x168], 0x0
0x7f3ceb7ff03c: jmp 0x7f3ceb7ff0f6
0x7f3ceb7ff041: nop
0x7f3ceb7ff042: nop
0x7f3ceb7ff043: nop
0x7f3ceb7ff044: nop
0x7f3ceb7ff045: nop
0x7f3ceb7ff046: nop
0x7f3ceb7ff047: nop
0x7f3ceb7ff048: nop
0x7f3ceb7ff049: nop
0x7f3ceb7ff04a: nop
0x7f3ceb7ff04b: nop
0x7f3ceb7ff04c: nop
0x7f3ceb7ff04d: nop
0x7f3ceb7ff04e: nop
0x7f3ceb7ff04f: nop
0x7f3ceb7ff050: push rbp
0x7f3ceb7ff051: mov rbp, rsp
0x7f3ceb7ff054: sub rsp, 0x70
0x7f3ceb7ff058: mov [rbp-0x60], r12
0x7f3ceb7ff05c: mov [rbp-0x58], rbx
0x7f3ceb7ff060: mov r11, 0x7f3d245783c0
0x7f3ceb7ff06a: mov r10, 0x7f3d245783c0
0x7f3ceb7ff074: cmp [r1